# Final Semantic Segmentation Source Code

## Setting up packages/libraries

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
# Install Gdown
!pip install gdown

# Install PyTorch
!conda install pytorch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0 cudatoolkit=11.3 -c pytorch
# Install mim
!pip install -U openmim
# Install mmengine
!mim install mmengine
# Install MMCV
!mim install 'mmcv >= 2.0.0rc1'

In [ ]:
# Install mmsegmentation
!rm -rf mmsegmentation
!git clone -b main https://github.com/open-mmlab/mmsegmentation.git
%cd mmsegmentation
!pip install -e .

In [ ]:
# Import google drive
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

In [ ]:
# Import pytorch
import torch, torchvision

# Import mmsegmentation
import mmseg
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset
from mmseg.apis import MMSegInferencer
from mmseg.utils import get_palette
from mmseg.models import build_segmentor
from mmseg.apis import init_model, inference_model, show_result_pyplot


# Import mmengine
import mmengine
from mmengine import Config
from mmengine.runner import Runner

# Import mmcv
import mmcv

# Import os
import os

# Import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Import numpy
import numpy as np

# Import PIL
from PIL import Image
import PIL

# Import zipfile
import zipfile

## Path Names

In [ ]:
# Path names for dataset
data_root_dir = "datasets/BoneScanDataset"
data_label_dir = os.path.join(data_root_dir, "labels")
data_image_dir = os.path.join(data_root_dir, "images")
data_split_dir = os.path.join(data_root_dir, "splits")

## Preparing New Dataset

In [ ]:
# Download Bone Scan Dataset
!gdown https://drive.google.com/uc?id=1yDYpCIJlx7rjnY0VfPcHMIg3ByD2JVA7
!unzip "BoneScanDatasetFinal.zip"
!mkdir datasets
!mv BoneScanDataset datasets/BoneScanDataset

In [ ]:
# Define palette and classes
classes = ('tengkorak', 'vertebra serviks', 'vertebra toraks', 'tulang rusuk', 'tulang dada', 'tulang selangka', 'tulang belikat', 'humerus', 'vertebra lumbalis', 'tulang kelangkang', 'panggul', 'tulang paha')
palette = [
      [176, 230,  13],
      [0, 151, 219],
      [126, 230, 225],
      [166,  55, 167],
      [230, 157, 180],
      [167, 110,  77],
      [121,   0,  24],
      [56,  65, 184],
      [230, 218,   0],
      [230, 114,  35],
      [13, 187,  62],
      [230, 182,  22],
]

In [ ]:
# Test reading an image
PIL.Image.open(os.path.join(data_image_dir, "0.png"))

In [ ]:
# Test reading an image
PIL.Image.open(os.path.join(data_image_dir, "1.png"))

In [ ]:
# Test reading a label
PIL.Image.open(os.path.join(data_label_dir, "0.png"))

In [ ]:
# Test reading a label
PIL.Image.open(os.path.join(data_label_dir, "1.png"))

In [ ]:
# Let's take a look at the segmentation map we got for an anterior label
img = Image.open(os.path.join(data_label_dir, "0.png"))
plt.figure(figsize=(8, 6))
im = plt.imshow(np.array(img.convert('RGB')))

# create a patch (proxy artist) for every color
patches = [mpatches.Patch(color=np.array(palette[i])/255,
                          label=classes[i]) for i in range(len(classes))]
# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,
           fontsize='large')

plt.show()

In [ ]:
# Let's take a look at the segmentation map we got for an posterior label
img = Image.open(os.path.join(data_label_dir, "0.png"))
plt.figure(figsize=(8, 6))
im = plt.imshow(np.array(img.convert('RGB')))

# create a patch (proxy artist) for every color
patches = [mpatches.Patch(color=np.array(palette[i])/255,
                          label=classes[i]) for i in range(len(classes))]
# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,
           fontsize='large')

plt.show()

## Run Inference with MMSeg trained weight

In [ ]:
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b0_8x1_1024x1024_160k_cityscapes/segformer_mit-b0_8x1_1024x1024_160k_cityscapes_20211208_101857-e7f88502.pth -P checkpoints

In [ ]:
# config_path = 'configs/segformer/segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py'
# checkpoint_path = 'checkpoints/segformer_mit-b0_8x1_1024x1024_160k_cityscapes_20211208_101857-e7f88502.pth'
# img_path = 'demo/demo.png'

# # build the model from a config file and a checkpoint file
# model = init_model(config_path, checkpoint_path, device='cuda:0')

# # inference on given image
# result = inference_model(model, img_path)

# # display the segmentation result
# vis_image = show_result_pyplot(model, img_path, result)

# PIL_image = Image.fromarray(vis_image.astype('uint8'), 'RGB')
# PIL_image

In [ ]:
# # Normalize every labels and turn it into segmentation map
# for item in os.listdir(os.path.join(data_anterior_dir, "masks")):
#   msk = mmcv.imread((os.path.join(data_anterior_dir, "masks", item)))
#   region = np.divide(msk[:,:,1], 21)
#   np.savetxt(os.path.join(data_anterior_dir, "masks", "{}.txt".format(item.split(".")[0])), region)

# for item in os.listdir(os.path.join(data_posterior_dir, "masks")):
#   msk = mmcv.imread((os.path.join(data_posterior_dir, "masks", item)))
#   region = np.divide(msk[:,:,1], 21)
#   np.savetxt(os.path.join(data_posterior_dir, "masks", "{}.txt".format(item.split(".")[0])), region)

In [ ]:
# # Check if the amount of png is the same as the txt
# counter1, counter2 = 0, 0
# for file in os.listdir(os.path.join(data_anterior_dir, "masks")):
#   if file.endswith(".txt"):
#     counter1 += 1
#   if file.endswith(".png"):
#     counter2 += 1
# print("Total amount of .txt files:", counter1)
# print("Total amount of .png files:", counter2)

In [ ]:
# # Test normalized segmentation maps
# seg_map = np.loadtxt(os.path.join(data_anterior_dir, "masks", "0.txt")).astype(np.uint8)
# seg_img = Image.fromarray(seg_map).convert('P')
# seg_img

## Splitting dataset for training, validation, and testing

In [ ]:
# mmcv.mkdir_or_exist(os.path.join(data_anterior_dir, "splits"))
# mmcv.mkdir_or_exist(os.path.join(data_posterior_dir, "splits"))

In [ ]:
# filename_list = [os.path.splitext(filename)[0] for filename in mmcv.scandir(
#     os.path.join(data_anterior_dir, "labels"), suffix='.png')]
# len(filename_list)

In [ ]:
# len(filename_list[33:])

In [ ]:
# # Split train/val/test set for anterior

# filename_list = [os.path.splitext(filename)[0] for filename in mmcv.scandir(
#     os.path.join(data_anterior_dir, "labels"), suffix='.png')]
# with open(os.path.join(data_anterior_dir, "splits", "train.txt"), 'w') as f:
#   # select first 4/5 as train set
#   train_length = int(len(filename_list)*4/5)
#   f.writelines(line + '\n' for line in filename_list[:train_length])
# with open(os.path.join(data_anterior_dir, "splits", "val.txt"), 'w') as f:
#   # select last 1/5 as val set
#   f.writelines(line + '\n' for line in filename_list[train_length:train_length+4])
# with open(os.path.join(data_anterior_dir, "splits", "test.txt"), 'w') as f:
#   # select last 1/5 as test set
#   f.writelines(line + '\n' for line in filename_list[train_length+4:])

In [ ]:
# # Split train/val/test set for posterior

# filename_list = [os.path.splitext(filename)[0] for filename in mmcv.scandir(
#     os.path.join(data_posterior_dir, "labels"), suffix='.png')]
# with open(os.path.join(data_posterior_dir, "splits", "train.txt"), 'w') as f:
#   # select first 4/5 as train set
#   train_length = int(len(filename_list)*4/5)
#   f.writelines(line + '\n' for line in filename_list[:train_length])
# with open(os.path.join(data_posterior_dir, "splits", "val.txt"), 'w') as f:
#   # select last 1/5 as val set
#   f.writelines(line + '\n' for line in filename_list[train_length:train_length+4])
# with open(os.path.join(data_posterior_dir, "splits", "test.txt"), 'w') as f:
#   # select last 1/5 as test set
#   f.writelines(line + '\n' for line in filename_list[train_length+4:])

## Experiment

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# # Download checkpoint iteration
# %cd /content/gdrive/MyDrive
# if ("pretrain" in os.listdir()) == False:
#   !mkdir pretrain
# %cd /content/mmsegmentation

In [ ]:
# Define global variables
BATCH_SIZE = 3
MAX_ITER = 6000
VAL_INTERVAL = 600
LOGGER_INTERVAL = 600
CHECKPOINT_INTERVAL = 3000
SEED = 777
DATASET_TYPE = 'BoneScanDataset'

In [ ]:
# Create custom dataset
@DATASETS.register_module()
class BoneScanDataset(BaseSegDataset):
  METAINFO = dict(classes = classes, palette = palette)
  def __init__(self, **kwargs):
    super().__init__(img_suffix='.png', seg_map_suffix='.png', **kwargs)


In [ ]:
# Make a directory to store trained_models
!mkdir trained_models

In [ ]:
# Make a directory to store segmented_imgs
!mkdir segmented_imgs

### Segformer

#### Segformer MIT-B0

In [ ]:
!mim download mmsegmentation --config segformer_mit-b0_8xb1-160k_cityscapes-1024x1024 --dest checkpoints

In [ ]:
# Download pretrained weight
!mkdir pretrain
!gdown https://drive.google.com/uc?id=1NgA2QDSvnBELZBScSChlfOt1sTXY5bjq
!mv mit_b0.pth pretrain/mit_b0.pth

In [ ]:
cfg = Config.fromfile('checkpoints/segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='SyncBN', requires_grad=True)
# cfg.crop_size = (128, 512)
# cfg.model.data_preprocessor.size = cfg.crop_size
# cfg.model.backbone.norm_cfg = cfg.norm_cfg
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# # modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = 'pretrain/mit_b0.pth'
# cfg.load_from = 'checkpoints/mit_b0.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_segformer_mit_b0 = cfg

In [ ]:
# Train the model with defined config
runner = Runner.from_cfg(cfg_segformer_mit_b0)
runner.train()

In [ ]:
  runner.val()

In [ ]:
runner.test()

In [ ]:
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/segformer_mit-b0_8xb1-160k_cityscapes-1024x1024_lr_0001"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

# Save checkpoint, config, and logs
shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py"), os.path.join(dst, "config.py"))
shutil.copy(os.path.join(src, "20230605_081951/vis_data/20230605_081951.json"), os.path.join(dst, "train.json"))
# shutil.copy(os.path.join(src, "20230605_044627/20230529_132858.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230605_081951/20230605_081951.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230605_081951/20230605_081951.log"), os.path.join(dst, "test.log"))
shutil.copy(os.path.join(src, "20230605_081951/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))



In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=1gMBBJPF1pmBaJzIT2-SNHM0Ra-CZPh6_
!unzip segformer_mit-b0_8xb1-160k_cityscapes-1024x1024_lr_0001-20230606T064002Z-001.zip
!mv segformer_mit-b0_8xb1-160k_cityscapes-1024x1024_lr_0001 trained_models/segformer_mit_b0

In [ ]:
%cd segmented_imgs
!mkdir segformer_mit_b0
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/segformer_mit_b0/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/segformer_mit_b0/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/segformer_mit_b0', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/segformer_mit_b0 /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/segformer_mit_b0

In [ ]:
# Zip folder
with zipfile.ZipFile('segformer_mit_b0_512channels.zip', 'w') as zip_file:
  zip_file.write('/content/mmsegmentation/segmented_imgs/segformer_mit_b0_512channels')

# Upload it into google drive
!cp segformer_mit_b0_512channels.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# img = mmcv.imread(os.path.join(data_image_dir, '51.png'))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))

#### Segformer MIT-B1

In [ ]:
!mim download mmsegmentation --config segformer_mit-b1_8xb1-160k_cityscapes-1024x1024 --dest checkpoints

In [ ]:
# Download pretrained weight
!mkdir pretrain
!gdown https://drive.google.com/uc?id=1V92bLc0agGL-gJp1JYutyArbVSS1GHk5
!mv mit_b1.pth pretrain/mit_b1.pth

In [ ]:
cfg = Config.fromfile('/content/mmsegmentation/checkpoints/segformer_mit-b1_8xb1-160k_cityscapes-1024x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='SyncBN', requires_grad=True)
# cfg.crop_size = (128, 512) # 65536
# # cfg.model.data_preprocessor.size = cfg.crop_size
# cfg.model.backbone.norm_cfg = cfg.norm_cfg
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# # # modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))


cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = '/content/mmsegmentation/checkpoints/segformer_mit-b1_8x1_1024x1024_160k_cityscapes_20211208_064213-655c7b3f.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_segformer_mit_b1 = cfg

In [ ]:
# Train the model with defined config
runner = Runner.from_cfg(cfg_segformer_mit_b1)
runner.train()

In [ ]:
runner.val()

In [ ]:
runner.test()

In [ ]:
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/segformer_mit_b1"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

# Save checkpoint, config, and logs
shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py"), os.path.join(dst, "segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py"))
shutil.copy(os.path.join(src, "20230529_132858/vis_data/20230529_133154.json"), os.path.join(dst, "train.json"))
shutil.copy(os.path.join(src, "20230529_132858/20230529_132858.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230529_133154/20230529_133154.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230529_133154/20230529_133154.log"), os.path.join(dst, "test.log"))
shutil.copy(os.path.join(src, "20230529_132858/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))

In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=12gudsvwKrDVbF6dX4k9Z4iv3UM7indNe
!unzip segformer_mit-b1_8xb1-160k_cityscapes-1024x1024_lr_0001-20230606T063952Z-001.zip
!mv segformer_mit-b1_8xb1-160k_cityscapes-1024x1024_lr_0001 trained_models/segformer_mit_b1

In [ ]:
%cd segmented_imgs
!mkdir segformer_mit_b1
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/segformer_mit_b1/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/segformer_mit_b1/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/segformer_mit_b1', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/segformer_mit_b1 /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/segformer_mit_b1

In [ ]:
# Zip folder
with zipfile.ZipFile('segformer_mit_b1_lr0001_512ch.zip', 'w') as zip_file:
  zip_file.write('/content/mmsegmentation/segmented_imgs/segformer_mit_b1_lr0001_512ch')

# Upload it into google drive
!cp segformer_mit_b1_lr0001_512ch.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# # Init the model from the config and the checkpoint
# checkpoint_path = './work_dirs/tutorial/iter_6000.pth'
# model = init_model(cfg_segformer_mit_b0, checkpoint_path, 'cuda:0')

# img = mmcv.imread(os.path.join(data_image_dir, "0.png"))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))

#### Segformer MIT-B0 512 Decoder Head Channels

In [ ]:
!mim download mmsegmentation --config segformer_mit-b0_8xb1-160k_cityscapes-1024x1024 --dest checkpoints

In [ ]:
# Download pretrained weight
!mkdir pretrain
!gdown https://drive.google.com/uc?id=1NgA2QDSvnBELZBScSChlfOt1sTXY5bjq
!mv mit_b0.pth pretrain/mit_b0.pth

In [ ]:
cfg = Config.fromfile('checkpoints/segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='SyncBN', requires_grad=True)
# cfg.crop_size = (128, 512)
# cfg.model.data_preprocessor.size = cfg.crop_size
# cfg.model.backbone.norm_cfg = cfg.norm_cfg
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# # modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13
cfg.model.decode_head.channels = 512

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = 'pretrain/mit_b0.pth'
# cfg.load_from = 'checkpoints/mit_b0.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_segformer_mit_b0 = cfg

In [ ]:
# Train the model with defined config
runner = Runner.from_cfg(cfg_segformer_mit_b0)
runner.train()

In [ ]:
  runner.val()

In [ ]:
runner.test()

In [ ]:
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/segformer_mit-b0_8xb1-160k_cityscapes-1024x1024_lr_0001"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

# Save checkpoint, config, and logs
shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py"), os.path.join(dst, "config.py"))
shutil.copy(os.path.join(src, "20230605_081951/vis_data/20230605_081951.json"), os.path.join(dst, "train.json"))
# shutil.copy(os.path.join(src, "20230605_044627/20230529_132858.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230605_081951/20230605_081951.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230605_081951/20230605_081951.log"), os.path.join(dst, "test.log"))
shutil.copy(os.path.join(src, "20230605_081951/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))



In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=1FMRhp_Y816lIl9UuxmEZVyO2SMJKWBxh
!unzip segformer_mit-b0_8xb1-160k_cityscapes-1024x1024_lr_0001_512-20230606T064002Z-001.zip
!mv segformer_mit_b0_lr_0001_512channels trained_models/segformer_mit_b0_512channels

In [ ]:
%cd segmented_imgs
!mkdir segformer_mit_b0_512channels
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/segformer_mit_b0_512channels/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/segformer_mit_b0_512channels/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/segformer_mit_b0_512channels', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/segformer_mit_b0_512channels /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/segformer_mit_b0_512channels

In [ ]:
# Zip folder
with zipfile.ZipFile('segformer_mit_b0_512channels.zip', 'w') as zip_file:
  zip_file.write('/content/mmsegmentation/segmented_imgs/segformer_mit_b0_512channels')

# Upload it into google drive
!cp segformer_mit_b0_512channels.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# img = mmcv.imread(os.path.join(data_image_dir, '51.png'))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))

#### Segformer MIT-B1 512 Decoder Head Channels

In [ ]:
!mim download mmsegmentation --config segformer_mit-b1_8xb1-160k_cityscapes-1024x1024 --dest checkpoints

In [ ]:
# Download pretrained weight
!mkdir pretrain
!gdown https://drive.google.com/uc?id=1V92bLc0agGL-gJp1JYutyArbVSS1GHk5
!mv mit_b1.pth pretrain/mit_b1.pth

In [ ]:
cfg = Config.fromfile('/content/mmsegmentation/checkpoints/segformer_mit-b1_8xb1-160k_cityscapes-1024x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='SyncBN', requires_grad=True)
# cfg.crop_size = (128, 512) # 65536
# # cfg.model.data_preprocessor.size = cfg.crop_size
# cfg.model.backbone.norm_cfg = cfg.norm_cfg
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# # # modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13
cfg.model.decode_head.channels = 512

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))


cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = '/content/mmsegmentation/checkpoints/segformer_mit-b1_8x1_1024x1024_160k_cityscapes_20211208_064213-655c7b3f.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_segformer_mit_b1 = cfg

In [ ]:
# Train the model with defined config
runner = Runner.from_cfg(cfg_segformer_mit_b1)
runner.train()

In [ ]:
runner.val()

In [ ]:
runner.test()

In [ ]:
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/segformer_mit_b1"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

# Save checkpoint, config, and logs
shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py"), os.path.join(dst, "segformer_mit-b0_8xb1-160k_cityscapes-1024x1024.py"))
shutil.copy(os.path.join(src, "20230529_132858/vis_data/20230529_133154.json"), os.path.join(dst, "train.json"))
shutil.copy(os.path.join(src, "20230529_132858/20230529_132858.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230529_133154/20230529_133154.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230529_133154/20230529_133154.log"), os.path.join(dst, "test.log"))
shutil.copy(os.path.join(src, "20230529_132858/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))

In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=1AyyHWH1qHWYwv1OdEITsq1lEaZ7y4j1O
!unzip segformer_mit-b1_8xb1-160k_cityscapes-1024x1024_lr_0001_512-20230606T063952Z-001.zip
!mv segformer_mit_b1_lr0001_512ch trained_models/segformer_mit_b1_512channels

In [ ]:
%cd segmented_imgs
!mkdir segformer_mit_b1_512channels
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/segformer_mit_b1_512channels/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/segformer_mit_b1_512channels/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/segformer_mit_b1_512channels', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/segformer_mit_b1_512channels /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/segformer_mit_b1_512channels

In [ ]:
# Zip folder
with zipfile.ZipFile('segformer_mit_b1_lr0001_512ch.zip', 'w') as zip_file:
  zip_file.write('/content/mmsegmentation/segmented_imgs/segformer_mit_b1_lr0001_512ch')

# Upload it into google drive
!cp segformer_mit_b1_lr0001_512ch.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# # Init the model from the config and the checkpoint
# checkpoint_path = './work_dirs/tutorial/iter_6000.pth'
# model = init_model(cfg_segformer_mit_b0, checkpoint_path, 'cuda:0')

# img = mmcv.imread(os.path.join(data_image_dir, "0.png"))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))

### Fully Convolutional Networks (FCN)


#### FCN ResNet18

In [ ]:
# Download config and checkpoints
!mim download mmsegmentation --config fcn_r18-d8_4xb2-80k_cityscapes-512x1024 --dest checkpoints

In [ ]:
cfg = Config.fromfile('checkpoints/fcn_r18-d8_4xb2-80k_cityscapes-512x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
# cfg.crop_size = (128, 512)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13
cfg.model.auxiliary_head.num_classes = 13

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = 'checkpoints/fcn_r18-d8_512x1024_80k_cityscapes_20201225_021327-6c50f8b4.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_fcn_r18 = cfg

In [ ]:
# Train the model with defined config
runner = Runner.from_cfg(cfg_fcn_r18)
runner.train()

In [ ]:
# Validate the model
runner.val()

In [ ]:
# Test the model
runner.test()

In [ ]:
# Save checkpoint, config, and logs
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/fcn_r18-d8_4xb2-80k_cityscapes-512x1024_lr_0001"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "fcn_r18-d8_4xb2-80k_cityscapes-512x1024.py"), os.path.join(dst, "config.py"))
shutil.copy(os.path.join(src, "20230605_091429/vis_data/20230605_091429.json"), os.path.join(dst, "train.json"))
shutil.copy(os.path.join(src, "20230605_091429/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))
# shutil.copy(os.path.join(src, "20230603_044957/20230603_044957.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230605_091429/20230605_091429.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230605_091429/20230605_091429.log"), os.path.join(dst, "test.log"))

In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=1u1nnbwv6b4J8Hpr9ROhIF0xJps7ug5hR
!unzip fcn_r18-d8_4xb2-80k_cityscapes-512x1024_lr_0001-20230607T143906Z-001.zip
!mv fcn_r18-d8_4xb2-80k_cityscapes-512x1024_lr_0001 trained_models/fcn_r18

In [ ]:
%cd segmented_imgs
!mkdir fcn_r18
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/fcn_r18/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/fcn_r18/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/fcn_r18', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/fcn_r18 /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/fcn_r18

In [ ]:
# Zip folder
with zipfile.ZipFile('fcn_r18.zip', 'w') as zip_file:
  zip_file.write('/content/mmsegmentation/segmented_imgs/fcn_r18')

# Upload it into google drive
!cp fcn_r18.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# # Init the model from the config and the checkpoint
# checkpoint_path = './work_dirs/tutorial/iter_10000.pth'
# model = init_model(cfg_fcn_r18_d8, checkpoint_path, 'cuda:0')

# img = mmcv.imread(os.path.join(data_image_dir, "0.png"))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))

#### FCN ResNet 50

In [ ]:
# Download config and checkpoints
!mim download mmsegmentation --config fcn_r50-d8_4xb2-80k_cityscapes-512x1024 --dest checkpoints

In [ ]:
cfg = Config.fromfile('checkpoints/fcn_r50-d8_4xb2-80k_cityscapes-512x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
# cfg.crop_size = (128, 512)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13
cfg.model.auxiliary_head.num_classes = 13

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = 'checkpoints/fcn_r50-d8_512x1024_80k_cityscapes_20200606_113019-03aa804d.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_fcn_r50 = cfg

In [ ]:
# Train the model with defined config
runner = Runner.from_cfg(cfg_fcn_r50)
runner.train()

In [ ]:
# Test the model
runner.test()

In [ ]:
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/fcn_r50-d8_4xb2-80k_cityscapes-512x1024_lr_0001"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

# Save checkpoint, config, and logs
shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "fcn_r50-d8_4xb2-80k_cityscapes-512x1024.py"), os.path.join(dst, "config.py"))
shutil.copy(os.path.join(src, "20230606_093027/vis_data/20230606_093027.json"), os.path.join(dst, "train.json"))
# shutil.copy(os.path.join(src, "20230605_044627/20230529_132858.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230606_093027/20230606_093027.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230606_093027/20230606_093027.log"), os.path.join(dst, "test.log"))
shutil.copy(os.path.join(src, "20230606_093027/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))

In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=1ecWyZVmN1XAOjSQvdtE6l4B_LaGRkXUp
!unzip fcn_r50-d8_4xb2-80k_cityscapes-512x1024_lr_0001-20230607T143917Z-001.zip
!mv fcn_r50-d8_4xb2-80k_cityscapes-512x1024_lr_0001 trained_models/fcn_r50

In [ ]:
%cd segmented_imgs
!mkdir fcn_r50
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/fcn_r50/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/fcn_r50/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/fcn_r50', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/fcn_r50 /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/fcn_r50

In [ ]:
# Zip folder
with zipfile.ZipFile('fcn_r50.zip', 'w') as zip_file:
  zip_file.write('/content/mmsegmentation/segmented_imgs/fcn_r50')

# Upload it into google drive
!cp fcn_r50.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# # Init the model from the config and the checkpoint
# checkpoint_path = './work_dirs/tutorial/iter_2500.pth'
# model = init_model(cfg_fcn_r50_d8, checkpoint_path, 'cuda:0')

# img = mmcv.imread(os.path.join(data_image_dir, "0.png"))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))

### Deeplab v3+

#### DeepLabv3+ ResNet 18

In [ ]:
# Download config and checkpoints
!mim download mmsegmentation --config deeplabv3plus_r18-d8_4xb2-80k_cityscapes-512x1024 --dest checkpoints

In [ ]:
cfg = Config.fromfile('checkpoints/deeplabv3plus_r18-d8_4xb2-80k_cityscapes-512x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
# cfg.crop_size = (128, 512)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13
cfg.model.auxiliary_head.num_classes = 13

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))

# cfg.custom_hooks = [dict(type='EmptyCacheHook', after_epoch=True, after_iter=True)]

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = 'checkpoints/deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_deeplab_r18 = cfg

In [ ]:
# Train the model
runner = Runner.from_cfg(cfg_deeplab_r18)
runner.train()

In [ ]:
# Test the model
runner.test()

In [ ]:
# Save checkpoint, config, and logs
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/deeplabv3plus_r18-d8_4xb2-80k_cityscapes-512x1024_lr_0001"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "deeplabv3plus_r18-d8_4xb2-80k_cityscapes-512x1024.py"), os.path.join(dst, "config.py"))
shutil.copy(os.path.join(src, "20230605_085525/vis_data/20230605_085525.json"), os.path.join(dst, "train.json"))
shutil.copy(os.path.join(src, "20230605_085525/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))
# shutil.copy(os.path.join(src, "20230603_044957/20230603_044957.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230605_085525/20230605_085525.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230605_085525/20230605_085525.log"), os.path.join(dst, "test.log"))

In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=1AMmeActlWmZenQWhSZk5_zXNIw11LTQE
!unzip deeplabv3plus_r18-d8_4xb2-80k_cityscapes-512x1024_lr_0001-20230606T064006Z-001.zip
!mv deeplabv3plus_r18-d8_4xb2-80k_cityscapes-512x1024_lr_0001 trained_models/deeplabv3plus_r18

In [ ]:
%cd segmented_imgs
!mkdir deeplabv3plus_r18
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/deeplabv3plus_r18/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/deeplabv3plus_r18/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/deeplabv3plus_r18', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/deeplabv3plus_r18 /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/deeplabv3plus_r18

In [ ]:
# # Zip folder
# with zipfile.ZipFile('deeplabv3plus_r18.zip', 'w') as zip_file:
#   zip_file.write('/content/mmsegmentation/segmented_imgs/deeplabv3plus_r18')

# # Upload it into google drive
# !cp deeplabv3plus_r18.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# # Init the model from the config and the checkpoint
# checkpoint_path = './work_dirs/tutorial/iter_2500.pth'
# model = init_model(cfg_deeplab_r18_d8, checkpoint_path, 'cuda:0')

# img = mmcv.imread(os.path.join(data_image_dir, "0.png"))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))


#### DeepLabv3+ ResNet 50

In [ ]:
# Download config and checkpoints
!mim download mmsegmentation --config deeplabv3plus_r50-d8_4xb2-80k_cityscapes-512x1024 --dest checkpoints

In [ ]:
cfg = Config.fromfile('checkpoints/deeplabv3plus_r50-d8_4xb2-80k_cityscapes-512x1024.py')
# Since we use only one GPU, BN is used instead of SyncBN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
# cfg.crop_size = (128, 512)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 13
cfg.model.auxiliary_head.num_classes = 13

# Modify dataset type and path
cfg.dataset_type = DATASET_TYPE
cfg.data_root = data_root_dir

cfg.train_dataloader.batch_size = BATCH_SIZE

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(256, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(256, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.0005)

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.001, weight_decay=0.0005),
    clip_grad=dict(max_norm=1, norm_type=2))

# cfg.custom_hooks = [dict(type='EmptyCacheHook', after_epoch=True, after_iter=True)]

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'

cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = dict(img_path="images", seg_map_path="labels")
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'

cfg.test_dataloader = cfg.val_dataloader
cfg.test_dataloader.dataset.ann_file = 'splits/test.txt'


# Load the pretrained weights
cfg.load_from = 'checkpoints/deeplabv3plus_r50-d8_512x1024_80k_cityscapes_20200606_114049-f9fb496d.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/tutorial'

cfg.train_cfg.max_iters = MAX_ITER
cfg.train_cfg.val_interval = VAL_INTERVAL
cfg.default_hooks.logger.interval = LOGGER_INTERVAL
cfg.default_hooks.checkpoint.interval = CHECKPOINT_INTERVAL

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=SEED)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

cfg_deeplab_r50 = cfg

In [ ]:
# Train the model
runner = Runner.from_cfg(cfg_deeplab_r50)
runner.train()

In [ ]:
# Test the model
runner.test()

In [ ]:
import shutil

src = "/content/mmsegmentation/work_dirs/tutorial"
dst = "/content/gdrive/MyDrive/pretrain/deeplabv3plus_r50-d8_4xb2-80k_cityscapes-512x1024_lr_0001"
if os.path.exists(dst) != True:
  os.makedirs(dst)
  print("Success on creating new dir.")

# Save checkpoint, config, and logs
shutil.copy(os.path.join(src, "iter_6000.pth"), os.path.join(dst, "iter_6000.pth"))
shutil.copy(os.path.join(src, "deeplabv3plus_r50-d8_4xb2-80k_cityscapes-512x1024.py"), os.path.join(dst, "config.py"))
shutil.copy(os.path.join(src, "20230606_064822/vis_data/20230606_064822.json"), os.path.join(dst, "train.json"))
# shutil.copy(os.path.join(src, "20230605_044627/20230529_132858.log"), os.path.join(dst, "train.log"))
shutil.copy(os.path.join(src, "20230606_064822/20230606_064822.json"), os.path.join(dst, "test.json"))
shutil.copy(os.path.join(src, "20230606_064822/20230606_064822.log"), os.path.join(dst, "test.log"))
shutil.copy(os.path.join(src, "20230606_064822/vis_data/scalars.json"), os.path.join(dst, "scalars.json"))

In [ ]:
# Download trained models
!gdown https://drive.google.com/uc?id=1D1Pc6JlIgqMKxC3nJJ8x7zzjqI9gMxbY
!unzip deeplabv3plus_r50-d8_4xb2-80k_cityscapes-512x1024_lr_0001-20230607T143317Z-001.zip
!mv deeplabv3plus_r50-d8_4xb2-80k_cityscapes-512x1024_lr_0001 trained_models/deeplabv3plus_r50

In [ ]:
%cd segmented_imgs
!mkdir deeplabv3plus_r50
%cd ..

with open(os.path.join(data_split_dir, 'test.txt'), 'r') as f:
    contents = f.read()
test_data = contents.split()


checkpoint_path ='/content/mmsegmentation/trained_models/deeplabv3plus_r50/iter_6000.pth'
model = init_model('/content/mmsegmentation/trained_models/deeplabv3plus_r50/config.py', checkpoint_path, 'cuda:0')

# Loop through the array of test dataset
for x in test_data:
  img = mmcv.imread(os.path.join(data_image_dir, '{}.png'.format(x)))
  result = inference_model(model, img)
  vis_result = show_result_pyplot(model, img, result)
  segmented_img = Image.fromarray(vis_result)
  segmented_img.save(os.path.join('/content/mmsegmentation/segmented_imgs/deeplabv3plus_r50', '{}.png'.format(x)))


In [ ]:
!cp -r segmented_imgs/deeplabv3plus_r50 /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs/deeplabv3plus_r50

In [ ]:
# Zip folder
with zipfile.ZipFile('deeplabv3plus_r50.zip', 'w') as zip_file:
  zip_file.write('/content/mmsegmentation/segmented_imgs/deeplabv3plus_r50')

# Upload it into google drive
!cp deeplabv3plus_r50.zip /content/gdrive/MyDrive/Tugas_Akhir/segmented_imgs

In [ ]:
# # Init the model from the config and the checkpoint
# checkpoint_path = './work_dirs/tutorial/iter_5000.pth'
# model = init_model(cfg_deeplab_r50_d8, checkpoint_path, 'cuda:0')

# img = mmcv.imread(os.path.join(data_image_dir, "0.png"))
# result = inference_model(model, img)
# plt.figure(figsize=(8, 6))
# vis_result = show_result_pyplot(model, img, result)
# plt.imshow(mmcv.bgr2rgb(vis_result))